In [1]:
# Import Libraries

import gym
import gym_anytrading
from gym_anytrading.envs import StocksEnv

from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3 import A2C, PPO
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.evaluation import evaluate_policy

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import timeit
import datetime
import time

import finta
TA = finta.TA
import investpy

from SBenv import TradingEnv, add_indicators, normalize_columns
from os.path import exists

df = pd.read_csv('./data/ASX.AX.csv')
coms = pd.read_csv('data/coms.csv', parse_dates=['Date'])
au_stocks = pd.read_csv('data/au_stocks.csv', parse_dates=['Date'])

In [2]:
# Add Indicators

df = add_indicators(df)
df = df.merge(coms, on='Date', how='left').merge(au_stocks, on='Date', how='left')
df.fillna(method='ffill', inplace=True)
df.dropna(inplace=True)
df.reset_index(inplace=True)
df = df.iloc[: , 1:]
df2 = normalize_columns(df)
df.to_csv('./data/DF_with_commodities.csv', index=False)
df2.to_csv('./data/Normalized_df_with_commodities.csv', index=False)

In [3]:
# Drop previous columns

df = pd.read_csv('./data/DF_with_commodities.csv')
df2 = pd.read_csv('./data/Normalized_df_with_commodities.csv')

au_stocks = ['rio', 'nab', 'wbc', 'fmg', 'tcl', 'all', 'wpl', 'sto', 'ncm', 'rea', 'shl', 'rhc', 'bxb', 'coh']

au_stocks = [i+'.ax' for i in au_stocks]
df.drop(au_stocks, axis=1, inplace=True)
df2.drop(au_stocks, axis=1, inplace=True)

In [4]:
df2

,Date,Close,Volume,Open,High,Low,SMA7,ADX,MI,ATR,...,bhp.ax,cba.ax,csl.ax,mqg.ax,anz.ax,brg.ax,qbe.ax,cpu.ax,wow.ax,tls.ax
0,0.000000,0.035294,0.006522,0.035294,0.034884,0.036145,0.027444,0.198734,0.700015,0.075949,...,0.010709,0.018220,0.020033,0.016625,0.015532,0.003440,0.154618,0.224927,0.010471,0.158416
1,0.000130,0.035294,0.008047,0.035294,0.034884,0.036145,0.027444,0.193733,0.697291,0.075949,...,0.009979,0.017973,0.020299,0.016682,0.015812,0.003557,0.155093,0.231904,0.008623,0.156256
2,0.000261,0.035294,0.031155,0.035294,0.034884,0.036145,0.027444,0.189089,0.662664,0.063291,...,0.012385,0.017397,0.020509,0.017096,0.016067,0.003557,0.150959,0.234189,0.008869,0.164894
3,0.000391,0.035294,0.021421,0.035294,0.034884,0.036145,0.027444,0.184777,0.618078,0.050633,...,0.012737,0.016936,0.020326,0.017340,0.017865,0.003322,0.153216,0.238634,0.004804,0.174251
4,0.000782,0.035294,0.033337,0.035294,0.034884,0.036145,0.027444,0.180773,0.589015,0.037975,...,0.013256,0.016375,0.019745,0.017096,0.014792,0.003440,0.149390,0.228323,0.003080,0.177131
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5323,0.998566,0.964706,0.007073,0.964706,0.953488,0.975904,0.972556,0.288133,0.651088,0.291139,...,0.800514,0.906511,0.875278,0.965210,0.945818,0.929585,0.545599,0.968685,0.866336,0.742998
5324,0.999218,0.964706,0.023785,0.964706,0.965116,0.987952,0.972556,0.291545,0.649227,0.253165,...,0.800724,0.919009,0.875911,0.979734,0.962906,0.952657,0.558916,0.981005,0.886714,0.753149
5325,0.999348,0.976471,0.015784,0.976471,0.976744,1.000000,0.974271,0.301129,0.644894,0.253165,...,0.808089,0.922650,0.874221,0.980470,0.960465,0.964193,0.556696,0.986652,0.886969,0.755687
5326,0.999479,0.976471,0.015042,0.976471,0.976744,0.987952,0.977702,0.296217,0.639560,0.253165,...,0.804301,0.910152,0.868848,0.970853,0.947852,0.954479,0.548928,0.981519,0.873978,0.750611


In [5]:
start = timeit.default_timer()

check_env(TradingEnv(df2, df)) 
env = DummyVecEnv([lambda: TradingEnv(df2, df)])

# Train the model
# model = A2C('MlpPolicy', env, verbose=0)
model = PPO('MlpPolicy', env, verbose=0) 
# model.set_env(env)
model.learn(total_timesteps=10**7)

stop = timeit.default_timer()

print('Time: ', datetime.timedelta(seconds = (stop - start)), " HH/MM/SS")  

KeyboardInterrupt: 

In [ ]:
model.save('./models/Second model')

In [27]:
mean_reward, std_reward = evaluate_policy(model, model.get_env(), n_eval_episodes=10)

/home/dev/anaconda3/envs/sbtrader/lib/python3.9/site-packages/stable_baselines3/common/evaluation.py:65: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Year:  0
Date:  2003-09-08
Last Networth:  1000
Networth:  1000.0
Yearly Return:  0.0 %
Avg Yearly Returns:  4 %

Year:  1
Date:  2004-09-02
Last Networth:  1000.0
Networth:  1038.0
Yearly Return:  3.8 %
Avg Yearly Returns:  4 %

Year:  2
Date:  2005-08-31
Last Networth:  1038.0
Networth:  1412.0
Yearly Return:  36.03 %
Avg Yearly Returns:  4 %

Year:  3
Date:  2006-08-29
Last Networth:  1412.0
Networth:  1616.0
Yearly Return:  14.45 %
Avg Yearly Returns:  4 %

Year:  4
Date:  2007-08-27
Last Networth:  1616.0
Networth:  2126.0
Yearly Return:  31.56 %
Avg Yearly Returns:  4 %

Year:  5
Date:  2008-08-22
Last Networth:  2126.0
Networth:  1684.0
Yearly Return:  -20.79 %
Avg Yearly Returns:  4 %

Year:  6
Date:  2009-08-19
Last Networth:  1684.0
Networth:  1752.0
Yearly Return:  4.04 %
Avg Yearly Returns:  4 %

Year:  7
Date:  2010-08-17
Last Networth:  1752.0
Networth:  1514.0
Yearly Return:  -13.58 %
Avg Yearly Returns:  4 %

Year:  8
Date:  2011-08-17
Last Networth:  1514.0
Networth:  

ValueError: could not broadcast input array from shape (0,) into shape (870,)

In [ ]:
obs = env.reset()
while True:
    action, _states = model.predict(obs)
    obs, rewards, dones, info = env.step(action)

In [6]:
df.shape[1]

44

In [23]:
step = 100
df.iloc[step-30:step]
# df.iloc[70:100,:]

,Date,Close,Volume,Open,High,Low,SMA7,ADX,MI,ATR,...,mqg.ax,anz.ax,shl.ax,brg.ax,qbe.ax,ncm.ax,cpu.ax,wbc.ax,wow.ax,tls.ax
70,2001-04-10,12.0,179082.0,12.0,13.0,12.0,12.000000,67.928639,30.620448,0.428571,...,11.128184,4.337886,3.923337,0.259361,4.881589,2.452670,4.350796,4.457866,4.156693,1.935092
71,2001-04-11,12.0,102130.0,12.0,13.0,12.0,12.000000,66.060553,31.862978,0.428571,...,11.082172,4.267991,3.894630,0.267466,5.010051,2.380105,4.182300,4.394450,4.269566,1.952318
72,2001-04-12,12.0,107142.0,12.0,12.0,12.0,12.000000,64.325902,32.763399,0.357143,...,11.005481,4.295887,3.919988,0.267466,4.855895,2.365591,4.152211,4.457526,4.348085,1.952318
73,2001-04-13,12.0,0.0,12.0,12.0,12.0,12.000000,62.715155,33.455725,0.285714,...,11.005481,4.295887,3.919988,0.267466,4.855895,2.365591,4.152211,4.457526,4.348085,1.952318
74,2001-04-16,12.0,0.0,12.0,12.0,12.0,12.000000,61.219461,31.121398,0.285714,...,11.005481,4.295887,3.919988,0.267466,4.855895,2.365591,4.152211,4.457526,4.348085,1.952318
75,2001-04-17,12.0,40139.0,12.0,12.0,12.0,12.000000,59.830602,29.883368,0.285714,...,10.825249,4.262664,3.889845,0.279623,4.830206,2.372849,4.052318,4.355242,4.284287,1.960931
76,2001-04-18,12.0,57631.0,12.0,12.0,12.0,12.000000,58.540947,29.097586,0.285714,...,10.775399,4.243858,4.086492,0.303938,4.783101,2.380105,3.851328,4.274779,4.318639,1.987919
77,2001-04-19,12.0,93658.0,12.0,12.0,12.0,12.000000,57.343411,28.549050,0.285714,...,11.174205,4.325349,3.971182,0.316096,4.723152,2.402600,3.911503,4.347059,4.343180,1.978157
78,2001-04-20,12.0,116374.0,12.0,12.0,12.0,12.000000,56.231413,28.142379,0.285714,...,11.072203,4.325349,3.899414,0.324201,4.675621,2.467182,3.901274,4.357286,4.338271,1.960931
79,2001-04-23,12.0,116275.0,12.0,12.0,12.0,12.000000,55.198843,26.514515,0.285714,...,11.059164,4.341021,3.827646,0.316096,4.603252,2.438156,3.995753,4.331034,4.328947,1.932220
